In [48]:
# Import packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import TFBertForSequenceClassification, BertTokenizer, DistilBertTokenizer, TFDistilBertForSequenceClassification
import tensorflow as tf

In [49]:
# Load in DataFrame
url = "./Data/Raw/Fake_News_Detection_340W.csv"
df = pd.read_csv(url, encoding = 'latin1')
df.head()
# df.drop(columns=["Unnamed: 0"])

# Sample an even number of rows for each label
# sample_size_per_label = 5000  

# # Separate the DataFrame into two based on the labels
# df_label_0 = df[df['label'] == 0]
# df_label_1 = df[df['label'] == 1]

# Sample from each of these DataFrames
# df_label_0_sample = df_label_0.sample(n=sample_size_per_label, random_state=1)
# df_label_1_sample = df_label_1.sample(n=sample_size_per_label, random_state=1)

# Concatenate the samples back into a single DataFrame
# df_sampled = pd.concat([df_label_0_sample, df_label_1_sample])

# # Shuffle the sampled DataFrame to mix the rows from the two labels
# df_sampled = df_sampled.sample(frac=1, random_state=1).reset_index(drop=True)



,News_Headline,Link_Of_News,Source,Stated_On,Date,Label
0,Says Osama bin Laden endorsed Joe Biden,https://www.politifact.com/factchecks/2020/jun...,Donald Trump Jr.,"June 18, 2020",6/19/2020,False
1,CNN aired a video of a toddler running away fr...,https://www.politifact.com/factchecks/2020/jun...,Donald Trump,"June 18, 2020",6/19/2020,False
2,Says Tim Tebow kneeled in protest of abortion...,https://www.politifact.com/factchecks/2020/jun...,Facebook posts,"June 12, 2020",6/19/2020,False
3,Even so-called moderate Democrats like Joe Bi...,https://www.politifact.com/factchecks/2020/jun...,Paul Junge,"June 10, 2020",6/19/2020,True
4,"""Our health department, our city and our count...",https://www.politifact.com/factchecks/2020/jun...,Jeanette Kowalik,"June 14, 2020",6/18/2020,True


In [50]:
# Test/train/validation split
train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)

Y_train = pd.DataFrame(train_data['Label']).astype(int)
X_train = train_data[['News_Headline', 'Source']]
Y_val = pd.DataFrame(val_data['Label']).astype(int)
X_val = val_data[['News_Headline', 'Source']]


print("Train data shape:", train_data.shape)
print("Validation_X data shape:", X_val.shape)
print("Validation_Y data shape:", Y_val.shape)

Train data shape: (7883, 6)
Validation_X data shape: (1971, 2)
Validation_Y data shape: (1971, 1)


In [51]:
# Load pre-trained BERT model and tokenizer
# bert_model_name = 'bert-base-uncased'
# tokenizer = BertTokenizer.from_pretrained(bert_model_name)
# model = TFBertForSequenceClassification.from_pretrained(bert_model_name)

In [52]:
disbert_tr = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizer.from_pretrained(disbert_tr)
model = TFDistilBertForSequenceClassification.from_pretrained(disbert_tr)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [53]:
max_length = 128

def tokenize_headlines(text_data, source_data):
    # Ensure all entries are converted to strings and concatenate them
    combined_data = text_data.astype(str) + " [SEP] " + source_data.astype(str)
    
    return tokenizer(
        combined_data.tolist(),  # Convert the combined data into a list of strings directly
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors='tf'
    )

# Assuming X_train and X_val DataFrames have 'News_Headline' and 'Source' columns
train_tokenized = tokenize_headlines(X_train['News_Headline'], X_train['Source'])
val_tokenized = tokenize_headlines(X_val['News_Headline'], X_val['Source'])


In [54]:
# combined_tokenized = {
#     'input_ids_title': title_tokenized['input_ids'],
#     'attention_mask_title': title_tokenized['attention_mask'],
#     'input_ids_text': text_tokenized['input_ids'],
#     'attention_mask_text': text_tokenized['attention_mask'],
# }

In [55]:
train_labels = Y_train['Label'].tolist()
val_labels = Y_val['Label'].tolist()

In [56]:
# Prepare TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_tokenized), train_labels)).shuffle(len(X_train)).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_tokenized), val_labels)).batch(32)

In [57]:
#Help find the best learning rate
#initial_learning_rate = 0.005
#decay_steps = len(train_dataset)  # Number of steps in one epoch
#decay_rate = 0.9

#def lr_scheduler(epoch, lr):
#    return lr * decay_rate

#scheduler_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

#Optimization and Evaluation
optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Fit the model with learning rate scheduling
model.fit(train_dataset, epochs=1)

# Save the model
model.save("bert_fine_tuned_model_2")

247/247 [==============================] - 1440s 6s/step - loss: 0.6985 - accuracy: 0.6224


INFO:tensorflow:Assets written to: bert_fine_tuned_model_2\assets


INFO:tensorflow:Assets written to: bert_fine_tuned_model_2\assets


In [60]:
loss, accuracy = model.evaluate(val_dataset)

print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)

62/62 [==============================] - 102s 2s/step - loss: 0.6664 - accuracy: 0.6164
Validation Loss: 0.6664489507675171
Validation Accuracy: 0.6164383292198181


In [59]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_dataset)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

NameError: name 'test_dataset' is not defined

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

# Define the hyperparameter distributions
param_dist = {
    'learning_rate': uniform(0.001, 0.01),
    'dropout_rate': uniform(0.1, 0.5),
    'num_layers': [1, 2, 3]
}

# Perform random search
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=10, cv=3, n_jobs=-1)
random_search_result = random_search.fit(X_train, Y_train)

# Summarize results
print("Best: %f using %s" % (random_search_result.best_score_, random_search_result.best_params_))


TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator <transformers.models.distilbert.modeling_tf_distilbert.TFDistilBertForSequenceClassification object at 0x0000024991DC1960> does not.